In [2]:
def deco(func):
    def inner():
        print 'running inner()'
    return inner

@deco
def target():
    print 'running target()'

In [3]:
target()  # 데코레이트된 함수 target()을 호출하면 inner()를 실행한다.

running inner()


In [4]:
target  # target 함수는 inner함수를 가리키고 있다.

<function __main__.inner>

데코레이터는 데코레이트된 함수를 다른 함수로 대체할 수 있다.

### 파이썬이 데코레이터를 실행하는 시점

데코레이터의 핵심 특징은 데코레이트된 함수가 정의된 직후에 바로 실행된다는 점이다.
이는 파이썬이 모듈을 로딩하는 시점, 즉 import 타임에 실행된다.

In [14]:
registry = []
def register(func):
    print 'running register. paramter is {}'.format(func)
    registry.append(func)
    return func

@register
def func1():
    print 'running func1()'
    
@register
def func2():
    print 'running func2()'
    
def func3():
    print 'running func3()'
    
def main():
    print 'running main()'
    print 'registry: {}'.format(registry)
    func1()
    func2()
    func3()

if __name__ == '__main__':
    main()

running register. paramter is <function func1 at 0x7fd17743e398>
running register. paramter is <function func2 at 0x7fd17743e6e0>
running main()
registry: [<function func1 at 0x7fd17743e398>, <function func2 at 0x7fd17743e6e0>]
running func1()
running func2()
running func3()


`register()` 데코레이터는 모듈에서 가장 먼저 실행된다. func1 ~ func3 함수는 `main`에서 명시적으로 호출될 때만 실행된다. <br>
위 코드를 `registration.py`모듈에 저장하고 모듈을 로딩하면,

In [16]:
import registration

running register. paramter is <function func1 at 0x7fd17743e398> <br>
running register. paramter is <function func2 at 0x7fd17743e6e0>

데코레이터는 모듈이 import 되자마자 실행되지만, 함수는 명시적으로 호출될 때만 실행됨을 알 수 있다.<br>
이는 파이썬에서 임포트타임과 런타임을 구분하는 이유이기도 하다. <br>
> `register()`의 경우 파라미터로 받은 함수를 그대로 리턴하지만, 보통의 데코레이터는 내부에서 새롭게 정의한 함수를 리턴한다.

### 변수 범위 규칙

In [20]:
b = 6  # 전역변수
def test_local_var(a):
    print a
    print b  # 6을 출력할까?
    b = 9
    

In [21]:
test_local_var(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

전역변수 `b`가 이미 정의되어 있기 때문에 `test_local_var`에서 6을 출력할 것으로 예상했지만 결과는 `UnboundLocalError`가 발생했다.<br>
왜 그럴까?

In [22]:
from dis import dis
dis(test_local_var)

  3           0 LOAD_FAST                0 (a)
              3 PRINT_ITEM          
              4 PRINT_NEWLINE       

  4           5 LOAD_FAST                1 (b)
              8 PRINT_ITEM          
              9 PRINT_NEWLINE       

  5          10 LOAD_CONST               1 (9)
             13 STORE_FAST               1 (b)
             16 LOAD_CONST               0 (None)
             19 RETURN_VALUE        


python3에서 디스어셈블 출력한 결과

In [24]:
  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_CONST               1 (6)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


SyntaxError: invalid syntax (<ipython-input-24-329d15ace98e>, line 1)

`10    LOAD_FAST`: 지역변수 `b`를 로딩한다. <br>
4번의 어셈블리어 코드를 보면 지역변수 `b`을 6으로 정의하는 부분이 나오지만, 컴파일러는 이미 이전에 `b`를 지역변수로 간주하고 있다. 

### 클로저

클로저가 뭔지 알아보기 전에 클래스를 이용해서 예제를 만들어보겠다. <br>
아래 예제는 새로운 숫자가 추가 될 때마다 추가된 숫자를 포함하여 숫자들의 평균을 계산한다.

In [32]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):  # 객체를 call할 때 호출되는 함수.
        self.series.append(new_value)
        total = sum(self.series)
        return total // len(self.series)

In [38]:
avg = Averager()
print avg(10)
print avg(11)
print avg(12)
# 파이썬2에서는 나누기 결과가 실수일 경우 내림으로 처리.

10
10
11


다음은 고차함수 `make_averager()`를 이용해서 예제를 구현한 것이다.

In [47]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/ len(series)
    return averager

In [48]:
avg = make_averager()
print avg(10)
print avg(12)
print avg(14)

10
11
12


`make_averager()` 함수 안의 `series`는 지역변수이므로 `make_averager`가 호출되고 `averager`를 리턴하고 나면 사라져야 마땅하다. <br>
그런데 `make_averager()`가 호출될때마다 이전에 숫자들이 추가된 `series`를 참조하여 숫자들의 평균을 구하게 된다. 왜 그럴까?

왜냐하면 `averager()` 안의 `series`는 **자유변수**기 때문이다.<br>
**자유변수는 지역범위에 바인딩 되어있지 않은 변수를 뜻한다.**

컴파일된 함수를 나타내는 `__code__`속성을 들여다보면 지역변수와 자유변수의 이름들을 알아낼 수 있다.

In [50]:
print avg.__code__.co_varnames  # 지역변수 이름들
print avg.__code__.co_freevars    # 자유변수 이름들

('new_value', 'total')
('series',)


`series`에 대한 바인딩은 반환된 `averager`함수의 `__closure__`속성에 리스트형태로 저장된다. <br>
`__closure__`의 각 엘리먼트는 `cell`객체이며, `cell`객체의 `cell_contents`속성에서 자유변수의 실제 값을 찾을 수 있다. <br>

> cells are special references to local variables of a parent scope, that follow the values those local variables point to. https://stackoverflow.com/questions/14413946/what-exactly-is-contained-within-a-obj-closure

위의 예제에 따르면 **클로저는 어떤 함수안에 정의된 함수며, 어떤 함수를 정의할 때 존재하던 자유변수의 바인딩을 유지해주는 함수라고 할 수 있다.**

### nonlocal 선언

아래의 `make_averager`는 잘못 정의되었다. 왜 그럴까?

In [51]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [52]:
avg = make_averager()
print avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`count`가 primitive type의 변수일 때 `count += 1`이 사실상 `count = count + 1`과 동일하기 때문에 문제가 발생한다. <br>
컴파일러가 `averager()`함수 내부의 `count`를 지역변수로 간주하기 때문이다. `total`도 마찬가지다. <br>
즉, `count`와 `total`은 자유변수가 아니므로 클로저에 저장되지 않는다.

이전 예제의 `series`경우에는 변수에 새로운 값을 정의하지 않고 `append()`, `len()`과 같이 메소드를 호출했기 때문에 문제가 되지 않았다. <br>
리스트가 가변형이라는 사실을 이용했을 뿐이다.

이 문제를 해결하기 위해 파이썬3은 `nonlocal`선언을 지원한다. <br>
`nonlocal`로 변수를 선언하면 함수 안에서 변수에 새로운 값을 할당하더라도 그 변수는 자유변수임을 나타낸다.
> 파이썬2는 `nonlocal`을 지원하지 않는 대신 다른 방법을 지원한다. https://www.python.org/dev/peps/pep-3104/ <br>
> 자유변수로 의도했던 변수를 가변 객체로 정의해서, 변경하고자 하는 값을 속성으로 갖게 함.